# **Q1 - Camera Relocalization**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import subprocess

# Data and project path
video_path = "/content/drive/MyDrive/3DCV/HW2/COLMAP/Sinica.mp4"
workspace_dir = "/content/drive/MyDrive/3DCV/HW2/COLMAP"

# Creating folders
frames_dir = os.path.join(workspace_dir, "frames")
database_dir = os.path.join(workspace_dir, "database")
output_dir = os.path.join(workspace_dir, "output")

os.makedirs(frames_dir, exist_ok=True)
os.makedirs(database_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

print(" Google Drive mounted successfully")
print(f"Video path: {video_path}")
print(f"Frames will be saved to: {frames_dir}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Google Drive mounted successfully
Video path: /content/drive/MyDrive/3DCV/HW2/COLMAP/Sinica.mp4
Frames will be saved to: /content/drive/MyDrive/3DCV/HW2/COLMAP/frames


**Installing COLMAP**

In [ ]:
# Install colmap
!apt-get install -y colmap ffmpeg

!colmap --version

print(" COLMAP and FFmpeg installed successfully")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
colmap is already the newest version (3.7-2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
ERROR: Command `--version` not recognized. To list the available commands, run `colmap help`.
 COLMAP and FFmpeg installed successfully


Extracting Frames fps=10

Totally 368 frames

In [ ]:
# Cheking Video
if os.path.exists(video_path):
    print(f" Video file found: {video_path}")

    # Extracting Frames FPS=10
    cmd = f'ffmpeg -i "{video_path}" -vf "fps=10" "{frames_dir}/frame_%04d.jpg" -hide_banner'
    !{cmd}


    frame_files = [f for f in os.listdir(frames_dir) if f.endswith('.jpg')]
    print(f" Extracted {len(frame_files)} frames to: {frames_dir}")


    if frame_files:
        print("Sample frames:", frame_files[:5])
else:
    print(f" Video file not found: {video_path}")

 Video file found: /content/drive/MyDrive/3DCV/HW2/COLMAP/Sinica.mp4
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/content/drive/MyDrive/3DCV/HW2/COLMAP/Sinica.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 0
    compatible_brands: isommp42
    creation_time   : 2025-10-08T09:15:37.000000Z
    com.android.version: 13
    com.android.manufacturer: Xiaomi
    com.android.model: 2201117TG
    com.xiaomi.product.marketname: Redmi Note 11
  Duration: 00:00:36.76, start: 0.000000, bitrate: 14341 kb/s
  Stream #0:0(eng): Video: h264 (High) (avc1 / 0x31637661), yuvj420p(pc, bt470bg/bt470bg/smpte170m), 1280x720, 13978 kb/s, 29.77 fps, 30.26 tbr, 90k tbn, 180k tbc (default)
    Metadata:
      rotate          : 90
      creation_time   : 2025-10-08T09:15:37.000000Z
      handler_name    : VideoHandle
      vendor_id       : [0][0][0][0]
    Side data:
      displaymatrix: rotation of -90.00 degrees
  Stream #0:1(eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 1

In [ ]:
    frame_files = [f for f in os.listdir(frames_dir) if f.endswith('.jpg')]
    print(f" Extracted {len(frame_files)} frames to: {frames_dir}")

 Extracted 368 frames to: /content/drive/MyDrive/3DCV/HW2/COLMAP/frames


In [ ]:
import os
os.environ['QT_QPA_PLATFORM'] = 'offscreen'

workspace_dir = "/content/drive/MyDrive/3DCV/HW2/COLMAP"
frames_dir = f"{workspace_dir}/frames"
database_path = f"{workspace_dir}/database/database.db"
output_dir = f"{workspace_dir}/output"

# Database folder
os.makedirs(f"{workspace_dir}/database", exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

print(" Environment setup completed")

 Environment setup completed


# **Feature Extraction**

In [ ]:

import os
# database_path = f"{workspace_dir}/database/database.db"
# if os.path.exists(database_path):
#     os.remove(database_path)
#     print(" Old database removed")

# Feature extracting
!colmap feature_extractor \
   --database_path "{database_path}" \
   --image_path "{frames_dir}" \
   --ImageReader.single_camera 1 \
   --SiftExtraction.use_gpu=false  # CPU

print(" Feature extraction with CPU completed")


Feature extraction

Processed file [1/368]
  Name:            frame_0001.jpg
  SKIP: Features for image already extracted.
Processed file [2/368]
  Name:            frame_0002.jpg
  SKIP: Features for image already extracted.
Processed file [3/368]
  Name:            frame_0003.jpg
  SKIP: Features for image already extracted.
Processed file [4/368]
  Name:            frame_0004.jpg
  SKIP: Features for image already extracted.
Processed file [5/368]
  Name:            frame_0005.jpg
  SKIP: Features for image already extracted.
Processed file [6/368]
  Name:            frame_0006.jpg
  SKIP: Features for image already extracted.
Processed file [7/368]
  Name:            frame_0007.jpg
  SKIP: Features for image already extracted.
Processed file [8/368]
  Name:            frame_0008.jpg
  SKIP: Features for image already extracted.
Processed file [9/368]
  Name:            frame_0009.jpg
  SKIP: Features for image already extracted.
Processed file [10/368]
  Name:            frame_001

In [ ]:
# # Matching the features
# database_path = f"{workspace_dir}/database/database.db"

# !colmap exhaustive_matcher \
#    --database_path "{database_path}" \
#    --SiftMatching.use_gpu=false  # CPU

# print("Feature matching with CPU completed")


# if os.path.exists(database_path):
#     new_size = os.path.getsize(database_path) / (1024*1024)
#     print(f" Database size after matching: {new_size:.1f} MB")

# Sequential Matcheing for 5 sequences of frames

In [ ]:

# Sequential Matcher
!colmap sequential_matcher \
   --database_path "{database_path}" \
   --SiftMatching.use_gpu=false \
   --SequentialMatching.overlap=5  # 5 sequential frames


Sequential feature matching

Matching image [1/368] in 0.003s
Matching image [2/368] in 0.001s
Matching image [3/368] in 0.001s
Matching image [4/368] in 0.001s
Matching image [5/368] in 0.001s
Matching image [6/368] in 0.001s
Matching image [7/368] in 0.001s
Matching image [8/368] in 0.001s
Matching image [9/368] in 0.001s
Matching image [10/368] in 0.001s
Matching image [11/368] in 0.001s
Matching image [12/368] in 0.001s
Matching image [13/368] in 0.001s
Matching image [14/368] in 0.001s
Matching image [15/368] in 0.001s
Matching image [16/368] in 0.001s
Matching image [17/368] in 0.001s
Matching image [18/368] in 0.001s
Matching image [19/368] in 0.001s
Matching image [20/368] in 0.001s
Matching image [21/368] in 0.001s
Matching image [22/368] in 0.001s
Matching image [23/368] in 0.001s
Matching image [24/368] in 0.001s
Matching image [25/368] in 0.001s
Matching image [26/368] in 0.000s
Matching image [27/368] in 0.000s
Matching image [28/368] in 0.001s
Matching image [29/368] in 

In [ ]:
import sqlite3
import pandas as pd

# conn = sqlite3.connect(database_path)
# conn.execute("DROP TABLE IF EXISTS matches;")
# conn.execute("DROP TABLE IF EXISTS two_view_geometries;")
# conn.commit()
# conn.close()
# print(" Removed tables")

conn = sqlite3.connect(database_path)
matches_count = pd.read_sql_query("SELECT COUNT(*) FROM matches;", conn).iloc[0,0]
print(f" # matches: {matches_count}")
conn.close()

 # matches: 2174


# **Triangulation for extracting 3D points cloud with estimated Camera positions**

In [ ]:

workspace_dir = "/content/drive/MyDrive/3DCV/HW2/COLMAP"
database_path = f"{workspace_dir}/database/database.db"
frames_dir = f"{workspace_dir}/frames"
sparse_dir = f"{workspace_dir}/output/sparse"


!colmap mapper \
    --database_path "{database_path}" \
    --image_path "{frames_dir}" \
    --output_path "{sparse_dir}"

Streaming output truncated to the last 5000 lines.

  => Continued observations: 847
  => Added observations: 468

Bundle adjustment report
------------------------
    Residuals : 15140
   Parameters : 1847
   Iterations : 11
         Time : 0.292666 [s]
 Initial cost : 0.50129 [px]
   Final cost : 0.492728 [px]
  Termination : Convergence

  => Merged observations: 77
  => Completed observations: 216
  => Filtered observations: 38
  => Changed observations: 0.038138

Bundle adjustment report
------------------------
    Residuals : 15452
   Parameters : 1862
   Iterations : 4
         Time : 0.12558 [s]
 Initial cost : 0.589428 [px]
   Final cost : 0.583637 [px]
  Termination : Convergence

  => Merged observations: 0
  => Completed observations: 5
  => Filtered observations: 3
  => Changed observations: 0.000927

Registering image #26 (279)

  => Image sees 836 / 3353 points

Pose refinement report
----------------------
    Residuals : 1676
   Parameters : 6
   Iterations : 8
     

In [ ]:
import os
workspace_dir = "/content/drive/MyDrive/3DCV/HW2/COLMAP"
database_path = f"{workspace_dir}/database/database.db"
frames_dir = f"{workspace_dir}/frames"
sparse_dir = f"{workspace_dir}/output/sparse"

# **Generating PLY Model format from the final results**

In [ ]:
# Check 3D model
if os.path.exists(f"{sparse_dir}/0"):
    print(" RECONSTRUCTION SUCCESSFUL!")

    # Analyzing final model
    print(" Analyze 3D Model:")
    !colmap model_analyzer --path "{sparse_dir}/0"

    # Convert to PLY
    print(" Convert to PLY...")
    !colmap model_converter \
        --input_path "{sparse_dir}/0" \
        --output_path "{workspace_dir}/output/model.ply" \
        --output_type PLY

    print(f"Final Model: {workspace_dir}/output/model.ply")

    # Final output file
    if os.path.exists(f"{workspace_dir}/output/model.ply"):
        file_size = os.path.getsize(f"{workspace_dir}/output/model.ply") / (1024*1024)
        print(f"PLY file was created: {file_size:.1f} MB")
    else:
        print("PLY file was not created")
else:
    print("No model in sparse/0/ !")

 RECONSTRUCTION SUCCESSFUL!
 Analyze 3D Model:
Cameras: 1
Images: 368
Registered images: 368
Points: 50183
Observations: 487509
Mean track length: 9.714624
Mean observations per image: 1324.752717
Mean reprojection error: 0.799155px
 Convert to PLY...
Final Model: /content/drive/MyDrive/3DCV/HW2/COLMAP/output/model.ply
PLY file was created: 0.7 MB


In [ ]:
!pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.7 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
%pip install open3d
!pip install open3d plotly

In [ ]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualize_point_cloud(ply_path):
    # Loding point cloud
    pcd = o3d.io.read_point_cloud(ply_path)

    print(f"Model Info:")
    print(f"   - # Points: {len(pcd.points):,}")
    print(f"   - Colors: {pcd.has_colors()}")
    print(f"   - Coordination Range:")

    points = np.asarray(pcd.points)
    if len(points) > 0:
        print(f"     X: {points[:,0].min():.2f} to {points[:,0].max():.2f}")
        print(f"     Y: {points[:,1].min():.2f} to {points[:,1].max():.2f}")
        print(f"     Z: {points[:,2].min():.2f} to {points[:,2].max():.2f}")

    # Interactive View
    print("Interactive View ...")
    o3d.visualization.draw_plotly([pcd])

    # Fixed View
    fig = plt.figure(figsize=(15, 5))

    #  (X-Z) View
    ax1 = fig.add_subplot(131)
    ax1.scatter(points[:,0], points[:,2], s=1, alpha=0.6)
    ax1.set_xlabel('X')
    ax1.set_ylabel('Z')
    ax1.set_title(' Top View (X-Z)')
    ax1.grid(True, alpha=0.3)
    ax1.axis('equal')

    # (X-Y) View
    ax2 = fig.add_subplot(132)
    ax2.scatter(points[:,0], points[:,1], s=1, alpha=0.6)
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_title('Front View (X-Y)')
    ax2.grid(True, alpha=0.3)
    ax2.axis('equal')

    # 3D View
    ax3 = fig.add_subplot(133, projection='3d')
    ax3.scatter(points[:,0], points[:,1], points[:,2], s=1, alpha=0.6)
    ax3.set_xlabel('X')
    ax3.set_ylabel('Y')
    ax3.set_zlabel('Z')
    ax3.set_title('3D View')

    plt.tight_layout()
    plt.show()

    return pcd

#  visualization
model_path = f"{workspace_dir}/output/model.ply"
point_cloud = visualize_point_cloud(model_path)

ModuleNotFoundError: No module named 'open3d'